<a href="https://colab.research.google.com/github/sbjglitch/Sanskrit2mal/blob/main/sanskrit_to_malayalam_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
%cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus

mal_lines = []
sanskrit_lines = []
with open ('bhagavadgita_malayalam.txt','r', encoding="utf8") as f:
  mal_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('ramayana_malayalam.txt','r', encoding="utf8") as f:
  mal_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('bible_malayalam2.txt','r', encoding="utf8") as f:
  mal_lines.extend([x.replace('\n', '') for x in f.readlines()])


with open ('bible_sanskrit.txt','r',encoding="utf8") as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('bhagvadgita_sanskrit.txt','r',encoding="utf8") as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('ramayan_sanskrit.txt','r',encoding="utf8") as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

print(len(mal_lines))
print(len(sanskrit_lines))

C:\Users\Glitch\Sanskrit2mal\parallel-corpus
18728
18728


In [39]:
import random
c = list(zip(sanskrit_lines, mal_lines))
#random.shuffle(c)

sanskrit_lines, mal_lines = zip(*c)

train_text_sa = sanskrit_lines[:18207]
train_text_mal = mal_lines[:18207]

dev_text_sa = sanskrit_lines[:701]
dev_text_mal = mal_lines[:701]

In [40]:
print(len(train_text_mal))
print(len(dev_text_mal))

print(len(train_text_sa))
print(len(dev_text_sa))

18207
701
18207
701


In [41]:
print(dev_text_mal[0])

ധൃതരാഷ്ട്രര്‍ ചോദിച്ചു: സഞ്ജയ, ധ‍ര്‍മ്മ ക്ഷേത്രമായ കുരുക്ഷേത്രത്തില്‍ ഒരുമിച്ചു ചേര്‍ന്നവരും യുദ്ധം ചെയ്യാന്‍ കൊതിക്കുന്നവരുമായ എന്റെ പുത്രന്‍മാരും പാണ്ഡവന്‍മാരും എന്ത് ചെയ്തു?


In [42]:
# Start and End tokens
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [43]:
# Reading the data and forming pairs
def readLangs(lang1, lang2):
    print("Reading lines...")
    
    # Forming pairs of sentences
    pairs = [[train_text_sa[i], train_text_mal[i]] for i in range(len(train_text_sa))]

    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [44]:
MAX_LENGTH = 200

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [45]:
def prepareData(lang1, lang2):
    input_lang, output_lang, pairs = readLangs(lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('sanskrit', 'mal')
print(random.choice(pairs)) # a random pair of parallel sentences

Reading lines...
Read 18207 sentence pairs
Trimmed to 18207 sentence pairs
Counting words...
Counted words:
sanskrit 82305
mal 67491
['कोटिं तत्र समुद्रे तु काञ्चनीं शतयोजनाम्। दुर्दर्शां  पारियात्रस्य गतां द्रक्ष्यथ वानराः।।', 'എന്നാൽ ഞാൻ നിങ്ങളോടു പറയുന്നു: നിങ്ങളുടെ ശത്രുക്കളെ സ്\u200cനേഹിക്കുവിൻ, നിങ്ങളെ ശപിക്കുന്നവരെ അനുഗ്രഹിക്കുവിൻ, നിങ്ങളെ വെറുക്കുന്നവർക്ക് നന്മ ചെയ്യുവിൻ, നിങ്ങളെ ഉപദ്രവിക്കുകയും ഉപദ്രവിക്കുകയും ചെയ്യുന്നവർക്കുവേണ്ടി പ്രാർത്ഥിക്കുക.']


In [46]:
print(input_lang.word2count['तथा'])

423


# **Seq2Seq Model**
---
A seq2seq network is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.

# **Encoder**

In [47]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# **Decoder**

In [48]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# **Training**


In [49]:
def indexesFromSentence(lang, sentence):
    res = []
    for word in sentence.split(' '):
      if word not in lang.word2index.keys():
        res.append(list(lang.word2index.values()))    
      else:
        res.append(lang.word2index[word])
    return res

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

# **Training the model**

In [50]:
teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [51]:
# Print time elapsed and estimated time remaining given the current time and progress %.
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [52]:
# Each iteration of training
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    #training_pairs = training_pairs1
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

# **Evaluation**

In [53]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [54]:
def evaluate_test(encoder, decoder):
  test = []
  %cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
  with open('test_sanskrit.sa', 'r', encoding="utf8") as f:
    test_text = [x.replace('\n', '') for x in f.readlines()]
  for sa_text in test_text:
    # print('>', pair[0])
    # print('=', pair[1])
    output_words, attentions = evaluate(encoder, decoder, sa_text)
    output_sentence = ' '.join(output_words)
    # print('<', output_sentence)
    # print('')
    test.append(output_sentence)
  with open('translated_malayalam.txt', 'w', encoding="utf8") as f:
    for line in test:
        f.write(line + '\n')

In [55]:
def translator(encoder, decoder):
    test_text = 'यत्र योगेश्वरः कृष्णो यत्र पार्थो धनुर्धरः। तत्र श्रीर्विजयो भूतिर्ध्रुवा नीतिर्मतिर्मम।।'

    output_words, attentions = evaluate(encoder, decoder, test_text)
    output_sentence = ' '.join(output_words)
    print(output_sentence)

In [56]:
 %cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal

C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal


In [ ]:
# defining hidden layer size and the encoder and decoder networks
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

# train
trainIters(encoder1, attn_decoder1, 75000, print_every=500)

0m 51s (- 127m 49s) (500 0%) 6.1959
1m 41s (- 125m 24s) (1000 1%) 6.7443
2m 34s (- 125m 53s) (1500 2%) 7.2950
3m 25s (- 125m 11s) (2000 2%) 7.1531
4m 17s (- 124m 27s) (2500 3%) 7.1707
5m 8s (- 123m 20s) (3000 4%) 7.2104
6m 0s (- 122m 39s) (3500 4%) 7.1455
6m 54s (- 122m 41s) (4000 5%) 7.2437
7m 46s (- 121m 45s) (4500 6%) 7.0816
8m 39s (- 121m 11s) (5000 6%) 7.2317
9m 29s (- 120m 0s) (5500 7%) 7.0103
10m 22s (- 119m 13s) (6000 8%) 7.1606
11m 15s (- 118m 34s) (6500 8%) 7.1006
12m 9s (- 118m 9s) (7000 9%) 7.1164
13m 2s (- 117m 22s) (7500 10%) 7.1365
13m 53s (- 116m 20s) (8000 10%) 6.8613
14m 45s (- 115m 27s) (8500 11%) 7.0905
15m 37s (- 114m 35s) (9000 12%) 7.0070
16m 28s (- 113m 33s) (9500 12%) 7.1389
17m 20s (- 112m 40s) (10000 13%) 7.0146
18m 12s (- 111m 51s) (10500 14%) 7.1187
19m 3s (- 110m 54s) (11000 14%) 7.0923
19m 55s (- 110m 0s) (11500 15%) 7.1323
20m 50s (- 109m 26s) (12000 16%) 7.1921
21m 43s (- 108m 38s) (12500 16%) 7.1462
22m 37s (- 107m 54s) (13000 17%) 7.2537
23m 30s (- 10

In [29]:
# saving the trained models
%cd C:\Users\Glitch\Sanskrit2mal
torch.save(encoder1.state_dict(), 'trained_encoder.pth')
torch.save(attn_decoder1.state_dict(), 'trained_decoder.pth')

C:\Users\Glitch\Sanskrit2mal


In [30]:
evaluate_test(encoder1, attn_decoder1)

C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal


In [31]:
%cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
output_text=[]
with open ('translated_malayalam.txt','r',encoding="utf8") as f:
  output_text.extend([f.readlines()])
print(output_text)

C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
[['അവൻ ഞാൻ രഥം നിങ്ങളെ ഞാൻ ഈ ഈ ഞാൻ പറഞ്ഞു: ഈ നീ <EOS>\n', 'യേശു അവനോടു: ഉത്തരം പറഞ്ഞു: യേശു പറഞ്ഞു: പറഞ്ഞു: <EOS>\n', 'ഹേ ലക്ഷ്മണാ, ഞാൻ ഈ ഈ ഈ നീ ഈ നിന്റെ നീ <EOS>\n', 'ഹേ ഞാൻ നീ നീ എന്നെ നീ <EOS>\n', 'ഹേ ഭരതാ, ഞാൻ തിളങ്ങുന്ന തിളങ്ങുന്ന തിളങ്ങുന്ന തിളങ്ങുന്ന ഞാൻ ഈ <EOS>\n', 'ശക്തനായ രാമൻ എന്റെ മകനെ ഈ ലോകത്ത് ഭരതൻ ഈ ലോകത്ത് <EOS>\n', 'അവൻ അവനോടു: അവനോടു: നിന്റെ തന്റെ ശക്തിയും ശക്തിയും ശക്തിയും ശക്തിയും ശക്തിയും പറഞ്ഞു: <EOS>\n', 'ഹേ രാമ, എന്റെ എന്റെ എന്റെ നിന്നെ കാണാൻ ഞാൻ നിന്നെ <EOS>\n', 'അപ്പോൾ അവർ എന്റെ എന്റെ എന്റെ എന്റെ എന്റെ <EOS>\n', "'അല്ലയോ സൗമ്യനായ നിന്ന് എന്റെ സഹോദരൻ അവിടെ നിന്ന് ഞാൻ <EOS>\n", 'അപ്പോൾ സ്ത്രീയേ, സ്ത്രീയേ, സ്ത്രീയേ, മൂന്ന് ലോകങ്ങളെയും ഈ രണ്ട് <EOS>\n', 'ഒരു ഒരു ഒരു പോലെ ഒരു ഒരു ഞാൻ ഒരു ഞാൻ അത് ഒരു <EOS>\n', 'ഹേ രാമാ, എന്റെ ഈ ഈ ഈ ഈ ഈ ഈ <EOS>\n', 'അപ്പോൾ അവനോടു: എന്റെ എന്റെ എന്റെ ഞാൻ ചെയ്തു, <EOS>\n', 'ഹേ ബ്രാഹ്മണേ, ഹേ രാമനെ അത് അത് അത് അത് എല്ലാ വിധത്തിലും ഞാൻ സത്യം ചെയ്യുന്നു. <EOS>\n', "'അല്ലയോ രാവണാ, ഒരു

In [33]:
translator(encoder1, attn_decoder1)

ഇപ്രകാരം അഭിസംബോധന പറഞ്ഞു: വാക്കുകൾ പറഞ്ഞു: ഈ രണ്ട് <EOS>


In [ ]:
%cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
import nltk
with open('test_malayalam.txt', 'r',  encoding="utf8") as f: # ground truth file
    reference = [x.replace('\n', '') for x in f.readlines()]

with open('translated_malayalam.txt', 'r',  encoding="utf8") as f: # translated Malayalam file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]

from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

bleu = nltk.bleu_score.corpus_bleu(reference, candidate, smoothing_function=smoothie )

print("bleu", bleu)

In [ ]:
score_ref_a = bleu.corpus_bleu([reference], [candidate])
print("1 document with 1 reference sentence: {}".format(score_ref_a))

In [37]:
from nltk.translate.bleu_score import sentence_bleu
with open('test_malayalam.txt', 'r',  encoding="utf8") as f: # ground truth file
    reference = [x.replace('\n', '') for x in f.readlines()]

with open('translated_malayalam.txt', 'r',  encoding="utf8") as f: # translated Malayalam file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]
    
print('BLEU score -> {}'sentence_bleu(reference, candidate ))

SyntaxError: invalid syntax (Temp/ipykernel_1372/1646575210.py, line 8)